# ◆[Backtest](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.buy) with OANDA API ver.H4
##### 目的：過去約2年の米ドル4時間足にて、バックテストを行う。高SQNスコアを目指す。
###### Create Date：2020/02/14　Author：M.Hasegawa
### ────────────────────────────────────────────────────────────────

#### 【table of contents】
- 0.[**Import module**](#Import_module)
- 1.[**Import data**](#Import_data)
- 2.[**Strategy**](#Strategy)
- 3.[**Backtest**](#Backtest)
- 4.[**Optimisation**](#Optimisation)

- ref:https://kernc.github.io/backtesting.py/doc/examples/Multiple%20Time%20Frames.html
- ref:https://saidataisei.hatenablog.com/entry/2019/10/13/003622
- ref:http://mmorley.hatenablog.com/entry/fx_backtesting01

## 0. Import module<a id='Import_module'></a>
- pip install backtesting
- pip install mpl_finance
- pip install oandapyV20
- pip install git+https://github.com/oanda/oandapy.git
- pip install TA_Lib-0.4.17-cp37-cp37m-win_amd64.whl

###### TA-LIB
- ref: https://qiita.com/ConnieWild/items/cb50f36425a683c914d2
- ref: https://www.lfd.uci.edu/~gohlke/pythonlibs/#ta-lib

In [1]:
%matplotlib inline
import oandapy
import pytz
import configparser
import pandas as pd
import numpy  as np
import talib as ta
from datetime         import datetime
from oandapyV20       import API
from backtesting      import Backtest, Strategy
from backtesting.lib  import crossover
from backtesting.lib  import resample_apply
from backtesting.test import SMA, GOOG
from pandas.core      import resample

config = configparser.ConfigParser()
config.read('./config_OANDA.txt')
account_id = config['oanda']['account_id']
api_key = config['oanda']['api_key']

import warnings
warnings.filterwarnings("ignore")

# ============================================================================
# Conv Japan Time
# ============================================================================
def iso_to_jp(iso):
    date = None
    try:
        date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%fZ')
        date = pytz.utc.localize(date).astimezone(pytz.timezone("Asia/Tokyo"))
    except ValueError:
        try:
            date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%f%z')
            date = date.astimezone(pytz.timezone("Asia/Tokyo"))
        except ValueError:
            pass
    return date

# ============================================================================
# Conv Format
# ============================================================================
def date_to_str(date):
    if date is None:
        return ''
    return date.strftime('%Y/%m/%d %H:%M:%S')

# ============================================================================
# SMA
# ============================================================================
def SMA(array, n):
    return pd.Series(array).rolling(n).mean()

# ============================================================================
# EMA
# ============================================================================
def EMA(array, n):
    return list(pd.Series(array).ewm(span=n).mean())

# ============================================================================
# RSI
# ============================================================================
def RSI(array, n):
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

# ============================================================================
# RCI:http://lowcost-greatidea.jp/technicalanalysis/rci/
# ============================================================================
def RCI(close, period):
    rank_period = np.arange(period, 0, -1)
    length = len(close)
    rci = np.zeros(length)
    
    for i in range(length):
        if i < period - 1:
            rci[i] = 0
        else :
            rank_price = pd.Series(close)[i - period + 1: i + 1].rank(method='min', ascending = False).values
            rci[i] = np.int32((1 - (6 * sum((rank_period - rank_price)**2)) / (period**3 - period)) * 100)
    return rci

# ============================================================================
# BB2
# ============================================================================
def bb2(array, n):
    gain=pd.DataFrame(array)
    gain.columns=['close']
    upper2, middle, lower2 = ta.BBANDS(gain.close, n,2,2,0)
    gain['bb_upper'] = upper2
    gain['bb_lower'] = lower2
    return gain['bb_upper'],gain['bb_lower']

# ============================================================================
# BB3
# ============================================================================
def bb3(array, n):
    gain=pd.DataFrame(array)
    gain.columns=['close']
    upper3, middle, lower3 = ta.BBANDS(gain.close, n,3,3,0)
    gain['bb_upper'] = upper3
    gain['bb_lower'] = lower3
    return gain['bb_upper'],gain['bb_lower']

# ============================================================================
# ADX
# ============================================================================
def adx(array,n):
    gain=pd.DataFrame(array)
    gain=gain.T
    gain.columns=['close','high','low']
    gain['adx'] = ta.ADX(gain['high'],gain['low'],gain['close'],n)
    return gain['adx']

C:\Users\arsma\Anaconda3\lib\site-packages\backtesting\_plotting.py:37: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# 1. Import data<a id='Import_data'></a>

URL:http://developer.oanda.com/rest-live/rates/

- M30・・・30 minute candlesticks, hour alignment
- H1・・・1 hour candlesticks, hour alignment
- H4・・・4 hour candlesticks, day alignment
- D・・・1 day candlesticks, day alignment
- W・・・1 week candlesticks, aligned to start of week
- M・・・1 month candlesticks, aligned to first day of the month

In [2]:
# API呼出
oa = oandapy.API(environment="live", access_token=api_key)

def df_init(data):
    df = pd.DataFrame(data["candles"])
    df['time'] = df['time'].apply(lambda x: iso_to_jp(x))   # 日本時間に変換
    df['time'] = df['time'].apply(lambda x: date_to_str(x)) # 形式変換（文字列型）
    df["time"] = pd.to_datetime(df["time"])                  # 型変換
    df = df[["time","openAsk","highAsk","lowAsk","closeAsk"]].copy()
    df = df.rename(columns={"openAsk":"Open","highAsk":"High","lowAsk":"Low","closeAsk":"Close"})
    df = df.set_index("time")
    return df

# 最大5000件
data = oa.get_history(instrument="USD_JPY", granularity='H4', count=5000)
df_5000 = df_init(data)

# 2. Strategy<a id='Strategy'></a>

## 2-1. Best of RCIStrategy

In [3]:
class RCIStrategy(Strategy):

    n1 = 21; n2 = 75
    rci_s = 9; rci_m = 24; rci_l = 52
    rsi_m = 14
    ema_s = 10
    
    def init(self):
        # SMA
        self.sma_s = self.I(SMA, self.data.Close, self.n1)
        self.sma_l = self.I(SMA, self.data.Close, self.n2)
        # RSI
        self.rsi_m = self.I(RSI, self.data.Close, self.rsi_m)
        # BB
        self.bb_u2,self.bb_l2 = self.I(bb2,self.data.Close,  self.n1)
        self.bb_u3,self.bb_l3 = self.I(bb3,self.data.Close,  self.n1)
        # RCI
        self.rci_s = self.I(RCI, self.data.Close, self.rci_s)
        self.rci_m = self.I(RCI, self.data.Close, self.rci_m)
        self.rci_l = self.I(RCI, self.data.Close, self.rci_l)
        # EMA
        self.ema_s = self.I(EMA, self.data.Close, self.ema_s)
        
    def next(self):
        
        price = self.data.Close[-1] # 前回の終値
        rci_s = self.rci_s[-1]
        rci_m = self.rci_m[-1]
        rci_l = self.rci_l[-1]
        
        # *******************************************************************************
        # ■ ロング
        # *******************************************************************************
        if (not self.position and ((1==0) 
             # -----------------------------------------------------------------------
             # ▼ Entry Long ※理想は3重底
             # -----------------------------------------------------------------------
             or (rci_s < -98 and rci_m < -90)
             or (rci_s < -82 and rci_m < 0 and rci_l > -71)
             or (rci_s < -63 and rci_m < -35 and rci_l > -75) 
            )):
            self.buy(sl=.982 * price) # ※ストップ：-200pips
            
        elif (self.position.is_short and ((1==0) 
               # -----------------------------------------------------------------------
               # ▼ Switch_Long ※ 保有中のショートを閉じてロングする
               # -----------------------------------------------------------------------
               or (rci_s < -87) 
               or (rci_s < -85 and rci_m > 75)
               or (rci_s < -75 and rci_l > 90)
               or (rci_s < -55 and rci_m > 84 and rci_l > 67)
             )):
            self.buy(sl=.982 * price) # ※ストップ：-200pips
            
        elif (self.position.is_long and ((1==0) 
               # -----------------------------------------------------------------------
               # ▼ Exit Long ※理想は3天井
               # -----------------------------------------------------------------------
               or (rci_m > 88)
               or (rci_l > 75)
               or (rci_s > 61 and rci_l > 27)
              )):
            self.position.close() 
            
        # *******************************************************************************
        # ■ ショート
        # *******************************************************************************
        if (not self.position and ((1==0) 
             # -----------------------------------------------------------------------
             # ▼ Entry Short ※理想は3天井
             # -----------------------------------------------------------------------
             or (rci_s < 98 and rci_s > 72 and rci_m > 36) # EntryShort1 修正
             or (rci_s > 70 and rci_l > 61)
             or (rci_s > 85 and rci_m < -50 and rci_l < -90) # 戻り売りの可能性
            )):
            self.sell(sl=1.012 * price) # ※ストップ：-130pips
            
        elif (self.position.is_long and ((1==0) 
               # -----------------------------------------------------------------------
               # ▼ Switch_Short ※ 保有中のロングを閉じてショートする
               # -----------------------------------------------------------------------
               or (rci_s > 95 and rci_m > 53)
               or (rci_s > -10 and crossover(self.sma_l, self.sma_s)) # デッドクロス(売りシグナル：短期下、長期上)
             )):
            self.sell(sl=1.012 * price) # ※ストップ：-130pips
            
        elif (self.position.is_short and ((1==0)
               # -----------------------------------------------------------------------
               # ▼ショートのExitルール(理想は3重底)
               # -----------------------------------------------------------------------
               or (rci_s < -91)
               or (rci_m < -86)
               or (rci_s < -87 and rci_m < -15)
               or (rci_s < -70 and rci_m < -52 and rci_l < -26)
              )):
            self.position.close()

## 2-2. Testing of RCIStrategy

In [64]:
class RCIStrategy2(Strategy):

    n1 = 21; n2 = 75
    rci_s = 9; rci_m = 24; rci_l = 52
    rsi_m = 14
    ema_s = 10
    
    def init(self):
        # SMA
        self.sma_s = self.I(SMA, self.data.Close, self.n1)
        self.sma_l = self.I(SMA, self.data.Close, self.n2)
        # RSI
        self.rsi_m = self.I(RSI, self.data.Close, self.rsi_m)
        # BB
        self.bb_u2,self.bb_l2 = self.I(bb2,self.data.Close,  self.n1)
        self.bb_u3,self.bb_l3 = self.I(bb3,self.data.Close,  self.n1)
        # RCI
        self.rci_s = self.I(RCI, self.data.Close, self.rci_s)
        self.rci_m = self.I(RCI, self.data.Close, self.rci_m)
        self.rci_l = self.I(RCI, self.data.Close, self.rci_l)
        # EMA
        self.ema_s = self.I(EMA, self.data.Close, self.ema_s)
        
    def next(self):
        
        price = self.data.Close[-1] # 前回の終値
        rci_s = self.rci_s[-1]
        rci_m = self.rci_m[-1]
        rci_l = self.rci_l[-1]
        
        # *******************************************************************************
        # ■ ロング
        # *******************************************************************************
        if (not self.position and ((1==0) 
             # -----------------------------------------------------------------------
             # ▼ Entry Long ※理想は3重底
             # -----------------------------------------------------------------------
             or (rci_s < -98 and rci_m < -90)
             or (rci_s < -82 and rci_m < 0 and rci_l > -71)
             or (rci_s < -63 and rci_m < -35 and rci_l > -75)  
            )):
            self.buy(sl=.982 * price) # ※ストップ：-200pips
            
        elif (self.position.is_short and ((1==0) 
               # -----------------------------------------------------------------------
               # ▼ Switch_Long ※ 保有中のショートを閉じてロングする
               # -----------------------------------------------------------------------
               or (rci_s < -87) 
               or (rci_s < -85 and rci_m > 75)
               or (rci_s < -75 and rci_l > 90)
               or (rci_s < -55 and rci_m > 84 and rci_l > 67)
             )):
            self.buy(sl=.982 * price) # ※ストップ：-200pips
            
        elif (self.position.is_long and ((1==0) 
               # -----------------------------------------------------------------------
               # ▼ Exit Long ※理想は3天井
               # -----------------------------------------------------------------------
               or (rci_m > 88)
               or (rci_l > 75)
               or (rci_s > 61 and rci_l > 27)
              )):
            self.position.close() 
            
        # *******************************************************************************
        # ■ ショート
        # *******************************************************************************
        if (not self.position and ((1==0) 
             # -----------------------------------------------------------------------
             # ▼ Entry Short ※理想は3天井
             # -----------------------------------------------------------------------
             or (rci_s < 98 and rci_s > 72 and rci_m > 36) # EntryShort1 修正
             or (rci_s > 70 and rci_l > 61)
             or (rci_s > 85 and rci_m < -50 and rci_l < -90) # 戻り売りの可能性
            )):
            self.sell(sl=1.012 * price) # ※ストップ：-130pips
            
        elif (self.position.is_long and ((1==0) 
               # -----------------------------------------------------------------------
               # ▼ Switch_Short ※ 保有中のロングを閉じてショートする
               # -----------------------------------------------------------------------
               or (rci_s > 95 and rci_m > 53)
               or (rci_s > 70 and rci_m < -73) # ★
               or (rci_s > -10 and crossover(self.sma_l, self.sma_s)) # デッドクロス(売りシグナル：短期下、長期上)
             )):
            self.sell(sl=1.012 * price) # ※ストップ：-130pips
            
        elif (self.position.is_short and ((1==0)
               # -----------------------------------------------------------------------
               # ▼ショートのExitルール(理想は3重底)
               # -----------------------------------------------------------------------
               or (rci_s < -91)
               or (rci_m < -86)
               or (rci_s < -87 and rci_m < -15)
               or (rci_s < -70 and rci_m < -52 and rci_l < -26)
              )):
            self.position.close()

# 3. Backtest<a id='Backtest'></a>
- [backtesting](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.buy)

#### H4×5000 = 約3年間 ※2020/2/14実施
- Equity Final($) = 1618.0
- Trades = 202
- Exposure(%) = 81.74
- Win Rate(%) = 72.77
- Return(%) = 61.8
- SQN = 5.11
- Sharpe Ratio =  0.33
- Sortino Ratio =  0.4
- Calmar Ratio =  0.05
- Avg. Trade Duration = 4 days 19:00:00

In [56]:
test_size = 5000
df_test = df_5000.tail(test_size)

In [65]:
bt = Backtest(df_test,
              RCIStrategy2,
              cash=1000,  # 所持金1000ドル(=約10万)
              commission=0.0002, # 取引手数料（為替価格に対する倍率で指定、為替価格100円でcommission=0.0002なら0.02円）
              trade_on_close=True # True：現在の終値に関してエントリー／ False：次の始値にエントリー
             )
out=bt.run()

#print('Duration=',out['Duration'])
print('- Equity Final($) =',round(out['Equity Final [$]'],2))
print('- Trades =',out['# Trades'])
print('- Exposure(%) =',round(out['Exposure [%]'],2))
print('- Win Rate(%) =',round(out['Win Rate [%]'],2))
print('- Return(%) =',round(out['Return [%]'],2))
print('- SQN =',round(out['SQN'],2))
print('- Sharpe Ratio = ',round(out['Sharpe Ratio'],2))
print('- Sortino Ratio = ',round(out['Sortino Ratio'],2))
print('- Calmar Ratio = ',round(out['Calmar Ratio'],2))
print('- Avg. Trade Duration =',out['Avg. Trade Duration'])
bt.plot()

- Equity Final($) = 1618.0
- Trades = 202
- Exposure(%) = 81.74
- Win Rate(%) = 72.77
- Return(%) = 61.8
- SQN = 5.11
- Sharpe Ratio =  0.33
- Sortino Ratio =  0.4
- Calmar Ratio =  0.05
- Avg. Trade Duration = 4 days 19:00:00


In [23]:
print(out)

Start                     2016-11-25 19:00:00
End                       2020-02-14 07:00:00
Duration                   1175 days 12:00:00
Exposure [%]                          82.4614
Equity Final [$]                      1544.18
Equity Peak [$]                       1545.22
Return [%]                            54.4178
Buy & Hold Return [%]                 2.77332
Max. Drawdown [%]                    -11.1963
Avg. Drawdown [%]                    -0.36397
Max. Drawdown Duration      476 days 04:00:00
Avg. Drawdown Duration        5 days 14:00:00
# Trades                                  184
Win Rate [%]                           73.913
Best Trade [%]                        2.14122
Worst Trade [%]                      -1.81964
Avg. Trade [%]                       0.238783
Max. Trade Duration          27 days 16:00:00
Avg. Trade Duration           5 days 07:00:00
Expectancy [%]                        0.59922
SQN                                    4.8258
Sharpe Ratio                      

|項目|項目の見方|
| :- | :- | 
|**Start**|ヒストリカルデータ開始日時|
|**End**|ヒストリカルデータ終了日時|
|**Duration**|ヒストリカルデータ期間|
|**Exposure (%)** |ポジション保有期間の割合|
|**Equity Final**|最終金額|
|**Equity Peak**|最高金額|
|**Return (%)**|利益率=損益÷開始時所持金×100|
|**Buy & Hold Return (%)**|((終了時の終値 開始時の終値)÷開始時の終値)の絶対値×100|
|**Max. Drawdown (%)**|最大下落率|
|**Avg. Drawdown (%)**|平均下落率|
|**Max. Drawdown Duration**|最大下落期間|
|**Avg. Drawdown Duration**|平均下落期間|
|**Trades**| 取引回数|
|**Win Rate (%)**|勝率=勝ち取引回数÷全取引回数×100|
|**Best Trade (%)**|1回の取引での利益の最大値÷所持金×100|
|**Worst Trade (%)**|1回の取引での損失の最大値÷所持金×100|
|**Avg. Trade (%)**|損益の平均値÷所持金×100|
|**Max. Trade Duration**|1回の取引での最長期間|
|**Avg. Trade Duration**|1回の取引での平均期間|
|**Expectancy (%)**|期待値=平均利益×勝率＋平均損失×敗率（１取引で期待できる利益、正は資産が増え、負は資産が減る。）|
|**SQN**|「期待値/標準偏差×√100」で計算されるシステムの品質の評価値。評価方法は、1.6～平均以下、2.0～平均、2.5～良い、3.0～優秀、5.1～最高、7.0～聖杯　※取引数が30以上で、SQN値は信頼できる。|
|**Sharpe Ratio**|シャープレシオとは利益とリスクの比率のことで、値が大きいほど資産曲線がなめらかになり安定性のある利益が見込めます。|
|**Sortino Ratio**|シャープレシオだけでは分からない下方リスクの抑制度合い。通常、この数値が大きいほど優れている（下落局面に強い）|
|**Calmar Ratio**|最大損失率に対する年間平均収益の比率 ※値が低いほど指定された期間に渡ってリスク調整ベースで実行された投資は悪化し、値が高いほどパフォーマンスが向上する。|

# 4. Optimisation<a id='optimisation'></a>

In [8]:
out2=bt.optimize(rci_s=range(9, 10),rci_m=range(24, 25),rci_l=range(52, 53))

In [9]:
print('▼最適化結果 ※',out2._trade_data.shape)
print('Equity Final($)=',round(out2['Equity Final [$]'],2))
print('Trades=',out2['# Trades'])
print('Win Rate(%)=',round(out2['Win Rate [%]'],2))
print('Return(%)=',round(out2['Return [%]'],2))
print('SQN=',round(out2['SQN'],2))
print('_strategy=',out2['_strategy'])

▼最適化結果 ※ (5000, 9)
Equity Final($)= 1544.18
Trades= 184
Win Rate(%)= 73.91
Return(%)= 54.42
SQN= 4.83
_strategy= RCIStrategy2(rci_s=9,rci_m=24,rci_l=52)


# 5. Output Excel Data

In [50]:
pd.set_option('display.max_rows', test_size)

xls_df = df_test
xls_df['RCI_s'] = RCI(xls_df.Close,9)
xls_df['RCI_m'] = RCI(xls_df.Close,24)
xls_df['RCI_l'] = RCI(xls_df.Close,52)
xls_df['SMA_s'] = SMA(xls_df.Close,21)
xls_df['SMA_l'] = SMA(xls_df.Close,75)

df_concat = pd.concat([out._trade_data.tail(test_size), xls_df], axis=1)
df_concat = df_concat.fillna({'SMA_s': 0.0})
df_concat = df_concat.fillna({'SMA_l': 0.0})

df_concat.to_excel("./backtest_USDJPY_" + str(test_size) +  ".xlsx",startrow=0, startcol=0)
display(df_concat.tail(5))

,Equity,Exit Entry,Exit Position,Entry Price,Exit Price,P/L,Returns,Drawdown,Drawdown Duration,Open,High,Low,Close,RCI_s,RCI_m,RCI_l,SMA_s,SMA_l
2020-02-13 15:00:00,1618.340410,NaN,NaN,NaN,NaN,NaN,NaN,0.015307,NaT,109.833,109.858,109.723,109.743,-26.0,55.0,71.0,109.857571,109.399640
2020-02-13 19:00:00,1618.134617,NaN,NaN,NaN,NaN,NaN,NaN,0.015432,NaT,109.742,109.799,109.625,109.757,-58.0,39.0,66.0,109.856000,109.409267
2020-02-13 23:00:00,1617.884726,NaN,NaN,NaN,NaN,NaN,NaN,0.015584,NaT,109.759,109.874,109.654,109.774,-78.0,29.0,60.0,109.855571,109.419120
2020-02-14 03:00:00,1616.943958,NaN,NaN,NaN,NaN,NaN,NaN,0.016156,NaT,109.773,109.857,109.751,109.838,-70.0,29.0,57.0,109.860524,109.432547
2020-02-14 07:00:00,1618.002322,NaN,NaN,NaN,NaN,NaN,NaN,0.015512,NaT,109.839,109.844,109.738,109.766,-63.0,19.0,51.0,109.861429,109.442267


# 6. Appendix VBA

Option Explicit

    Private entry_dic, exit_dic, win_entry_dic, lse_entry_dic, win_exit_dic, lse_exit_dic As Object
    Private COL_RCI_S, COL_RCI_M, COL_RCI_L, COL_SMA_S, COL_SMA_L As Integer
    
Public Sub CreateTradeResult()
    'Application.ScreenUpdating = False
    
'01.Time
'02.Equity
'03.Exit Entry
'04.Exit Position
'05.Entry Price
'06.Exit Price
'07.P/L
'08.Returns
'09.Drawdown
'10.Drawdown Duration
'11.Open
'12.High
'13.Low
'14.Close
'15.rci_s
'16.rci_m
'17.rci_l
'18.SMA_s
'19.SMA_l
'-------------------
'20.新規Entry
'21.ポジション
'22.結果
'23.建値
'24.Entry条件
'25.獲得pips
'26.損失pips
'27.Exit条件
'28.保有pips
'29.保有日数
'30.Trade No
'31.獲得pips(※)
'32.損失pips(※)

    ' 設定クリア
    ActiveSheet.Cells.ClearFormats
    ActiveSheet.Columns("T:AZ").ClearContents ' ★列変更時
    ActiveSheet.Columns("A:A").Font.Bold = False
    ActiveSheet.Rows("1:1").Font.Bold = True
    
    ' ヘッダ表示
    ActiveSheet.Range("A1").Value = "Time"
    
    ActiveSheet.Range("T1").Value = "新規Entry"
    ActiveSheet.Range("U1").Value = "ポジション"
    ActiveSheet.Range("V1").Value = "結果"
    ActiveSheet.Range("W1").Value = "建値"
    ActiveSheet.Range("X1").Value = "Entry時条件"
    ActiveSheet.Range("Y1").Value = "獲得pips"
    ActiveSheet.Range("Z1").Value = "損失pips"
    ActiveSheet.Range("AA1").Value = "Exit条件"
    ActiveSheet.Range("AB1").Value = "保有pips"
    ActiveSheet.Range("AC1").Value = "保有日数"
    ActiveSheet.Range("AD1").Value = "Trade No"
    ActiveSheet.Range("AE1").Value = "獲得pips(※)"
    ActiveSheet.Range("AF1").Value = "損失pips(※)"
    
    ActiveSheet.Columns("A:A").NumberFormatLocal = "yy-mm-dd aaa hh"
    ActiveSheet.Columns("B:B").NumberFormatLocal = "0.000"
    ActiveSheet.Columns("I:I").NumberFormatLocal = "0.000"
    ActiveSheet.Columns("G:G").NumberFormatLocal = "0.000"
    ActiveSheet.Columns("K:N").NumberFormatLocal = "0.000"
    ActiveSheet.Columns("R:S").NumberFormatLocal = "0.000"
    ActiveSheet.Columns("AA:AA").NumberFormatLocal = "0.00"
    
    Dim i, clr_no, trade_no, win_pips, lse_pips As Integer
    Dim row_min, row_max, col_max, rci_s, rci_m, rci_l As Integer
    Dim last_position, last_trade_no, last_entry_p, last_entry_logic, last_entry_row, result As String
    Dim sma_s, sma_l As Double
    Dim position, s_trade_no, entry_p, exit_p, rtn_val, entry_logic, exit_logic, close_p As String
    Dim last_pos_long, last_pos_short As Boolean
    
    Call entry_dic_init
    Call exit_dic_init
    
    row_min = 2
    row_max = ActiveSheet.Cells(1, 1).End(xlDown).row
    col_max = ActiveSheet.Cells(1, 1).End(xlToRight).Column
    
    last_entry_p = "0"
    last_entry_logic = ""
    last_entry_row = "0"
    last_position = ""
    last_trade_no = 0
    trade_no = 0
    
    COL_RCI_S = 15
    COL_RCI_M = 16
    COL_RCI_L = 17
    COL_SMA_S = 18
    COL_SMA_L = 19
    
    
    For i = row_min To row_max
        
        '初期化
        entry_logic = ""
        exit_logic = ""
        position = ""
        s_trade_no = ""
        
        win_pips = 0
        lse_pips = 0
        
        entry_p = ActiveSheet.Cells(i, 5).Value
        exit_p = ActiveSheet.Cells(i, 6).Value
        rtn_val = ActiveSheet.Cells(i, 8).Value
        close_p = ActiveSheet.Cells(i, 14).Value
        rci_s = CInt(ActiveSheet.Cells(i, COL_RCI_S).Value)
        rci_m = CInt(ActiveSheet.Cells(i, COL_RCI_M).Value)
        rci_l = CInt(ActiveSheet.Cells(i, COL_RCI_L).Value)
        sma_s = CDbl(ActiveSheet.Cells(i, COL_SMA_S).Value)
        sma_l = CDbl(ActiveSheet.Cells(i, COL_SMA_L).Value)
        
        last_pos_long = InStr(UCase(last_position), "LONG") > 0
        last_pos_short = InStr(UCase(last_position), "SHORT") > 0
        
        ' ※行の種類
        ' 1.エントリーのみ ---------------> 判断可能
        ' 2.決済のみ       ---------------> 判断可能
        ' 3.エントリーと決済       -------> 判断可能
        ' 4.保有中     -------------------> エントリー時にポジションを記載することで判断
        ' 5.保有なし   -------------------> 上記以外
        
        '********* debug用 *********
        If last_trade_no = 23 Then
            Debug.Print "▼" + Str(i) + "行目 T" + CStr(last_trade_no)
        End If
        '***************************
        
        ' ===============================================
        ' ■ エントリー情報表示
        ' ===============================================
        If (entry_p <> "" And rtn_val = "") Or (entry_p <> "" And rtn_val <> "") Then
            ' ▼ 1.エントリーのみ + ▼ 3.エントリーと決済
            trade_no = trade_no + 1
            s_trade_no = CStr(trade_no)
            entry_logic = GetEntryName(i, rci_s, rci_m, rci_l, last_position, sma_s, sma_l)
            position = entry_logic + "保有"
            
            ActiveSheet.Cells(i, 20).Value = entry_logic
            ActiveSheet.Cells(i, 21).Value = position
            ActiveSheet.Cells(i, 30).Value = "T" + CStr(trade_no)
            
        ElseIf entry_p = "" And rtn_val <> "" Then
            ' ▼ 2.決済のみ
            ActiveSheet.Cells(i, 20).Value = entry_logic
            ActiveSheet.Cells(i, 21).Value = position
            ActiveSheet.Cells(i, 30).Value = "T" + CStr(last_trade_no)
            
        ElseIf last_pos_long Or last_pos_short Then
            ' ▼ 4.保有中
            ActiveSheet.Cells(i, 20).Value = entry_logic
            ActiveSheet.Cells(i, 21).Value = last_position
            ActiveSheet.Cells(i, 30).Value = "T" + CStr(last_trade_no)
            ActiveSheet.Cells(i, 28).Value = IIf(last_pos_long, CInt((CDbl(close_p) - CDbl(last_entry_p)) * 100), CInt((CDbl(last_entry_p) - CDbl(close_p)) * 100))
            
        Else
            ' ▼ 5.保有なし
            ActiveSheet.Cells(i, 20).Value = "-"
            ActiveSheet.Cells(i, 21).Value = "-"
            ActiveSheet.Cells(i, 30).Value = "-"
        End If
                
        ' ===============================================
        ' ■ 決済情報表示
        ' ===============================================
        If rtn_val <> "" Then
            
            
            If CDbl(rtn_val) > 0 Then
                ' 勝ち
                If last_pos_long Then
                    result = "○win_long"
                    win_pips = Int((exit_p - last_entry_p) * 100)
                    clr_no = 37 ' 薄い青
                Else
                    result = "○win_short"
                    win_pips = Int((last_entry_p - exit_p) * 100)
                    clr_no = 38 ' 薄い赤
                End If
                
                ' entry_logicに値があれば、同時エントリー（前回Exitがswitchかを判断）
                exit_logic = GetExitName(i, last_pos_long, win_pips, lse_pips, entry_logic, rci_s, rci_m, rci_l)
            
                win_entry_dic(last_entry_logic) = win_entry_dic(last_entry_logic) + 1
                win_exit_dic(exit_logic) = win_exit_dic(exit_logic) + 1
                
            Else
                ' 負け
                If last_pos_long Then
                    result = "●lose_long"
                    lse_pips = Int((exit_p - last_entry_p) * 100) ' 損失pips
                    clr_no = 23 ' 青
                Else
                    result = "●lose_short"
                    lse_pips = Int((last_entry_p - exit_p) * 100) ' 損失pips
                    clr_no = 3 '赤
                End If
                
                ' entry_logicに値があれば、同時エントリー（前回Exitがswitchかを判断）
                exit_logic = GetExitName(i, last_pos_long, win_pips, lse_pips, entry_logic, rci_s, rci_m, rci_l)
            
                lse_entry_dic(last_entry_logic) = lse_entry_dic(last_entry_logic) + 1
                lse_exit_dic(exit_logic) = lse_exit_dic(exit_logic) + 1
                
            End If ' rtn_val > 0

            ActiveSheet.Cells(i, 22).Value = result
            ActiveSheet.Cells(i, 23).Value = last_entry_p
            ActiveSheet.Cells(i, 24).Value = last_entry_logic
            ActiveSheet.Cells(i, 25).Value = win_pips
            ActiveSheet.Cells(i, 26).Value = lse_pips
            ActiveSheet.Cells(i, 27).Value = exit_logic
            ActiveSheet.Cells(i, 29).Value = (i - last_entry_row) * 4 / 24  '保有日数
            ActiveSheet.Cells(i, 29).NumberFormatLocal = "0.00"
            
            ActiveSheet.Cells(last_entry_row, 31).Value = win_pips
            ActiveSheet.Cells(last_entry_row, 32).Value = lse_pips
            
            ' 書式設定
            ActiveSheet.Range(Cells(last_entry_row, 20), Cells(i, 21)).Interior.ColorIndex = clr_no 'エントリー行
            ActiveSheet.Range(Cells(i, 22), Cells(i, col_max)).Interior.ColorIndex = clr_no      '決済行
            
            If InStr(exit_logic, "Loss") > 0 Then
                'ロスカット文字色変更
                ActiveSheet.Range(Cells(last_entry_row, 20), Cells(i, 21)).Font.ColorIndex = 6 'エントリー行
                ActiveSheet.Range(Cells(i, 22), Cells(i, col_max)).Font.ColorIndex = 6      '決済行
                ActiveSheet.Range(Cells(i, 22), Cells(i, col_max)).Font.Bold = True
            End If
            
            
        End If ' rtn_val <> ""
        
        ' 過去データ更新
        If entry_p <> "" Then
            ' ▼ エントリーのみ + ▼エントリーと決済
            last_position = position
            last_entry_p = entry_p
            last_entry_logic = entry_logic
            last_entry_row = i
            last_trade_no = s_trade_no ' 前回トレード番号更新
            
        ElseIf entry_p = "" And rtn_val <> "" Then
            ' ▼決済のみ
            last_position = ""
            last_entry_p = 0
            last_entry_logic = ""
            last_entry_row = -1
            last_trade_no = 0
        End If
        
    Next i
    
    Call ShowAggregate
    
    ' ==============================================================
    ' 体裁
    ' ==============================================================
    ActiveSheet.Range(Cells(1, 1), Cells(row_max, col_max)).Borders.LineStyle = True
    ActiveSheet.Range(Cells(1, 1), Cells(row_max, col_max)).Font.Size = 10
    ActiveSheet.Range(Cells(1, 1), Cells(row_max, col_max)).Font.Name = "ＭＳ ゴシック"
    ActiveSheet.Range(Cells(1, 1), Cells(1, col_max)).Interior.ColorIndex = 34
    
    ' 列幅設定
    Cells.EntireColumn.AutoFit
    
    'ヘッダ固定
    Range("B2").Select
    ActiveWindow.FreezePanes = True
    
    Columns("I:M").Hidden = True
    
    Set entry_dic = Nothing
    Set exit_dic = Nothing
    Set win_entry_dic = Nothing
    Set lse_entry_dic = Nothing
    Set win_exit_dic = Nothing
    Set lse_exit_dic = Nothing
    
    Application.Goto Reference:=Range("T5005"), Scroll:=True
    Application.ScreenUpdating = False
        
End Sub

'# *******************************************************************************
'# ■ エントリー条件
'# *******************************************************************************
Private Sub entry_dic_init()
    Set entry_dic = CreateObject("Scripting.Dictionary")
    Set win_entry_dic = CreateObject("Scripting.Dictionary")
    Set lse_entry_dic = CreateObject("Scripting.Dictionary")
    
    ' ▼ ●●●● ▼ エントリーロング ▼ ●●●● ▼
    entry_dic.Add "Entry_Long1", "通常ロング1"
    entry_dic.Add "Entry_Long2", "通常ロング2"
    entry_dic.Add "Entry_Long3", "通常ロング3"
    ' Switch
    entry_dic.Add "Switch_Long1", "切替ロング1"
    entry_dic.Add "Switch_Long2", "切替ロング2"
    entry_dic.Add "Switch_Long3", "切替ロング3"
    entry_dic.Add "Switch_Long4", "切替ロング4"
        
    ' -----------------------------------------------
    
    ' ▼ ■■■■ ▼ エントリーショート ▼ ■■■■ ▼
    entry_dic.Add "Entry_Short1", "通常ショート1"
    entry_dic.Add "Entry_Short2", "通常ショート2"
    entry_dic.Add "Entry_Short3", "通常ショート3"
    ' Switch
    entry_dic.Add "Switch_Short1", "切替ショート1"
    entry_dic.Add "Switch_Short2", "切替ショート2"
    entry_dic.Add "Switch_Short3", "切替ショート3"
    ' -----------------------------------------------
    Dim curKey As Variant
    For Each curKey In entry_dic
        win_entry_dic.Add curKey, 0 '初期化
        lse_entry_dic.Add curKey, 0 '初期化
    Next
    
End Sub

Private Function GetEntryName(row, rci_s, rci_m, rci_l, position, sma_s, sma_l) As String
    Dim rtn As String
    Dim pos_is_short, pos_is_long As Boolean
    rtn = ""
    pos_is_long = InStr(UCase(position), "LONG") > 0
    pos_is_short = InStr(UCase(position), "SHORT") > 0
    
    If 1 = 0 Then
    ' ▼ ●●●● ▼ エントリーロング ▼ ●●●● ▼
    ' Switch ※ 決済直後の通常エントリーの場合、前回保有情報が消えていないため、先にSwitchロジックを記載する。
    ElseIf pos_is_short And rci_s < -87 Then: rtn = "Switch_Long1"
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
    
    ElseIf pos_is_short And rci_s < -85 And rci_m > 75 Then: rtn = "Switch_Long2"  ' 押し目買いチャンス
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_M).Interior.ColorIndex = 3 '赤
    
    ElseIf pos_is_short And rci_s < -75 And rci_l > 90 Then: rtn = "Switch_Long3"  ' 押し目買いチャンス
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_L).Interior.ColorIndex = 3 '赤
    
    ElseIf pos_is_short And rci_s < -55 And rci_m > 84 And rci_l > 67 Then: rtn = "Switch_Long4"   ' 押し目買いチャンス
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_M).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_L).Interior.ColorIndex = 3 '赤
        
    ' 通常
    ElseIf rci_s < -98 Then: rtn = "Entry_Long1"
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        
    ElseIf rci_s < -82 And rci_m < -0 And rci_l > -71 Then: rtn = "Entry_Long2"
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_M).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_L).Interior.ColorIndex = 3 '赤
        
    ElseIf rci_s < -63 And rci_m < -35 And rci_l > -75 Then: rtn = "Entry_Long3"
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_M).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_L).Interior.ColorIndex = 3 '赤
        
    ' -----------------------------------------------
    End If
    
    If 1 = 0 Then
    ' ▼ ■■■■ ▼ エントリーショート ▼ ■■■■ ▼
    ' Switch ※ 決済直後の通常エントリーの場合、前回保有情報が消えていないため、先にSwitchロジックを記載する。
    ElseIf pos_is_long And rci_s > 95 And rci_m > 53 Then: rtn = "Switch_Short1"
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_M).Interior.ColorIndex = 3 '赤
        
    ElseIf pos_is_long And rci_s > 70 And rci_m < -73 Then: rtn = "Switch_Short2"
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_M).Interior.ColorIndex = 3 '赤
        
    ElseIf pos_is_long And rci_s > -10 And sma_s < sma_l Then: rtn = "Switch_Short3"
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_SMA_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_SMA_L).Interior.ColorIndex = 3 '赤
        
    ' 通常
    ElseIf rci_s < 98 And rci_s > 72 And rci_m > 36 Then: rtn = "Entry_Short1"
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_M).Interior.ColorIndex = 3 '赤
        
    ElseIf rci_s > 70 And rci_l > 61 Then: rtn = "Entry_Short2"
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_L).Interior.ColorIndex = 3 '赤
        
    ElseIf rci_s > 85 And rci_m < -50 And rci_l < -90 Then: rtn = "Entry_Short3" ' 戻り売りの可能性
        ActiveSheet.Cells(row, COL_RCI_S).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_M).Interior.ColorIndex = 3 '赤
        ActiveSheet.Cells(row, COL_RCI_L).Interior.ColorIndex = 3 '赤
    ' -----------------------------------------------
    End If
    
    If rtn = "" Then
        MsgBox ("Entry Logic Error.")
    End If
    
    GetEntryName = rtn
    
End Function

'# *******************************************************************************
'# ■ エクジット条件
'# *******************************************************************************
Private Sub exit_dic_init()
    Set exit_dic = CreateObject("Scripting.Dictionary")
    Set lse_exit_dic = CreateObject("Scripting.Dictionary")
    Set win_exit_dic = CreateObject("Scripting.Dictionary")

    ' ▼ ◆◆◆◆ ▼ エクジットロング ▼ ◆◆◆◆ ▼
    exit_dic.Add "Exit_Long1", "通常エクジットロング1"
    exit_dic.Add "Exit_Long2", "通常エクジットロング2"
    exit_dic.Add "Exit_Long3", "通常エクジットロング3"
    ' Switch
    exit_dic.Add "Switch_Short1", "ショート切替1"
    exit_dic.Add "Switch_Short2", "ショート切替2"
    exit_dic.Add "Switch_Short3", "ショート切替3"
    exit_dic.Add "★Loss_cut_Long", "ロスカット強制終了"
    ' -----------------------------------------------
    
    ' ▼ ★★★★ ▼ エクジットショート ▼ ★★★★ ▼
    exit_dic.Add "Exit_Short1", "通常エクジットショート1"
    exit_dic.Add "Exit_Short2", "通常エクジットショート2"
    exit_dic.Add "Exit_Short3", "通常エクジットショート3"
    exit_dic.Add "Exit_Short4", "通常エクジットショート4"
    ' Switch
    exit_dic.Add "Switch_Long1", "ロング切替"
    exit_dic.Add "Switch_Long2", "ロング切替"
    exit_dic.Add "Switch_Long3", "ロング切替"
    exit_dic.Add "Switch_Long4", "ロング切替"
    exit_dic.Add "★Loss_cut_Short", "ロスカット強制終了"
    ' -----------------------------------------------
    Dim curKey As Variant
    For Each curKey In exit_dic
        win_exit_dic.Add curKey, 0 '初期化
        lse_exit_dic.Add curKey, 0 '初期化
    Next
    
End Sub

Private Function GetExitName(row, long_flg, win_pips, lse_pips, entry_logic, rci_s, rci_m, rci_l) As String
    Dim rtn As String
    Dim switch_flg, re_entry_flg As Boolean
    rtn = ""
    
    switch_flg = InStr(entry_logic, "Switch") > 0
    re_entry_flg = InStr(entry_logic, "ReEntry") > 0
    
    If long_flg Then
        If 1 = 0 Then
        ' ▼ ◆◆◆◆ ▼ エクジットロング ▼ ◆◆◆◆ ▼
        ElseIf switch_flg Then: rtn = entry_logic
        ElseIf re_entry_flg Then: rtn = entry_logic
        '通常
        ElseIf rci_m > 88 Then: rtn = "Exit_Long1"
            ActiveSheet.Cells(row, COL_RCI_M).Font.ColorIndex = 32
            
        ElseIf rci_l > 75 Then: rtn = "Exit_Long2"
            ActiveSheet.Cells(row, COL_RCI_L).Font.ColorIndex = 32
            
        ElseIf rci_s > 61 And rci_l > 27 Then: rtn = "Exit_Long3"
            ActiveSheet.Cells(row, COL_RCI_S).Font.ColorIndex = 32
            ActiveSheet.Cells(row, COL_RCI_L).Font.ColorIndex = 32

        'ロスカット
        ElseIf lse_pips < -100 Then: rtn = "★Loss_cut_Long"
        Else:
        End If
    
    Else
        If 1 = 0 Then
        ' ▼ ★★★★ ▼ エクジットショート ▼ ★★★★ ▼
        ElseIf switch_flg Then: rtn = entry_logic
        ElseIf re_entry_flg Then: rtn = entry_logic
        '通常
        ElseIf rci_s < -91 Then: rtn = "Exit_Short1"
            ActiveSheet.Cells(row, COL_RCI_S).Font.ColorIndex = 32

        ElseIf rci_m < -86 Then: rtn = "Exit_Short2"
            ActiveSheet.Cells(row, COL_RCI_M).Font.ColorIndex = 32
        
        ElseIf rci_s < -87 And rci_m < -15 Then: rtn = "Exit_Short3"
            ActiveSheet.Cells(row, COL_RCI_S).Font.ColorIndex = 32
            ActiveSheet.Cells(row, COL_RCI_M).Font.ColorIndex = 32
        
        ElseIf rci_s < -70 And rci_m < -52 And rci_l < -26 Then: rtn = "Exit_Short4"
            ActiveSheet.Cells(row, COL_RCI_S).Font.ColorIndex = 32
            ActiveSheet.Cells(row, COL_RCI_M).Font.ColorIndex = 32
            ActiveSheet.Cells(row, COL_RCI_L).Font.ColorIndex = 32
        
        'ロスカット
        ElseIf lse_pips < -100 Then: rtn = "★Loss_cut_Short"
        Else:
    ' -----------------------------------------------
        End If
    
    End If
                    
    If rtn = "" Then
        rtn = "★ Error : Not Found Exit Name ★"
    End If
    
    GetExitName = rtn
    
End Function

Private Sub ShowAggregate()

    Dim curKey As Variant
    Dim row_idx, k, win_cnt, lose_cnt As Integer
    Dim StartCell As Range
    
    ' ==============================================================
    ' Entry Logic
    ' ==============================================================
    Set StartCell = ActiveSheet.Range("T5005")
    row_idx = 1
    StartCell.Offset(1, 0).Value = "▼ Entry Logic": StartCell.Offset(1, 1).Value = "回数"
    StartCell.Offset(1, 2).Value = "勝": StartCell.Offset(1, 3).Value = "負": StartCell.Offset(1, 4).Value = "勝率"
    
    For Each curKey In entry_dic
        row_idx = row_idx + 1
        StartCell.Offset(row_idx, 0).Value = curKey
        StartCell.Offset(row_idx, 1).Formula = "=COUNTIF($T$2:$T$5001, " + CStr(StartCell.Offset(row_idx, 0).Address) + ")"
    Next
    For Each curKey In win_entry_dic
        For k = 2 To entry_dic.Count + 1
            If curKey = StartCell.Offset(k, 0).Value Then
                win_cnt = CInt(win_entry_dic.Item(curKey))
                lose_cnt = CInt(lse_entry_dic.Item(curKey))
                StartCell.Offset(k, 2).Value = win_cnt
                StartCell.Offset(k, 3).Value = lose_cnt
                If Not (win_cnt = 0 And lose_cnt = 0) Then
                    StartCell.Offset(k, 4).Value = win_cnt / (win_cnt + lose_cnt) * 100
                    StartCell.Offset(k, 4).NumberFormatLocal = "0.0"
                End If
            End If
        Next k
    Next
    
    ActiveSheet.Range(StartCell.Offset(1, 0), StartCell.Offset(1, 4)).Interior.ColorIndex = 34
    ActiveSheet.Range(StartCell.Offset(1, 0), StartCell.Offset(entry_dic.Count + 1, 4)).Borders.LineStyle = True
        
    ' ==============================================================
    ' Exit Logic
    ' ==============================================================
    Set StartCell = ActiveSheet.Range("T5025")
    row_idx = 1
    StartCell.Offset(1, 0).Value = "▼ Exit Logic": StartCell.Offset(1, 1).Value = "回数"
    StartCell.Offset(1, 2).Value = "勝": StartCell.Offset(1, 3).Value = "負": StartCell.Offset(1, 4).Value = "勝率"
    
    For Each curKey In exit_dic
        row_idx = row_idx + 1
        StartCell.Offset(row_idx, 0).Value = curKey
        StartCell.Offset(row_idx, 1).Formula = "=COUNTIF($AA$2:$AA$5001, " + CStr(StartCell.Offset(row_idx, 0).Address) + ")"
    Next
    For Each curKey In win_exit_dic
        For k = 2 To exit_dic.Count + 1
            If curKey = StartCell.Offset(k, 0).Value Then
                win_cnt = CInt(win_exit_dic.Item(curKey))
                lose_cnt = CInt(lse_exit_dic.Item(curKey))
                StartCell.Offset(k, 2).Value = win_cnt
                StartCell.Offset(k, 3).Value = lose_cnt
                If Not (win_cnt = 0 And lose_cnt = 0) Then
                    StartCell.Offset(k, 4).Value = win_cnt / (win_cnt + lose_cnt) * 100
                    StartCell.Offset(k, 4).NumberFormatLocal = "0.0"
                End If
            End If
        Next k
    Next
    
    ActiveSheet.Range(StartCell.Offset(1, 0), StartCell.Offset(1, 4)).Interior.ColorIndex = 34
    ActiveSheet.Range(StartCell.Offset(1, 0), StartCell.Offset(exit_dic.Count + 1, 4)).Borders.LineStyle = True

    ' ==============================================================
    ' 勝敗結果
    ' ==============================================================
    Set StartCell = ActiveSheet.Range("AC5005")
    StartCell.Offset(1, 0).Value = "▼結果"
    
    StartCell.Offset(2, 0).Value = "○win_long"
    StartCell.Offset(3, 0).Value = "○win_short"
    StartCell.Offset(4, 0).Value = "●lose_long"
    StartCell.Offset(5, 0).Value = "●lose_short"
    
    StartCell.Offset(1, 1).Value = "回数"
    StartCell.Offset(2, 1).Value = "=COUNTIF($V$2:$V$5001,""○win_long"")"
    StartCell.Offset(3, 1).Value = "=COUNTIF($V$2:$V$5001,""○win_short"")"
    StartCell.Offset(4, 1).Value = "=COUNTIF($V$2:$V$5001,""●lose_long"")"
    StartCell.Offset(5, 1).Value = "=COUNTIF($V$2:$V$5001,""●lose_short"")"
    
    StartCell.Offset(1, 2).Value = "発生確率"
    StartCell.Offset(2, 2).Value = "=AB5007/SUM($AD$5007:$AB$5010)"
    StartCell.Offset(3, 2).Value = "=AB5008/SUM($AD$5007:$AB$5010)"
    StartCell.Offset(4, 2).Value = "=AB5009/SUM($AD$5007:$AB$5010)"
    StartCell.Offset(5, 2).Value = "=AB5010/SUM($AD$5007:$AB$5010)"
    
    ActiveSheet.Range(StartCell.Offset(1, 0), StartCell.Offset(1, 2)).Interior.ColorIndex = 34
    ActiveSheet.Range(StartCell.Offset(1, 0), StartCell.Offset(5, 2)).Borders.LineStyle = True
    ActiveSheet.Range(StartCell.Offset(1, 2), StartCell.Offset(5, 2)).NumberFormatLocal = "0.00"

End Sub
